## Word2Vec 4Lakh words, dim 300 and Vec2Word pt files 

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
resnet152 = models.resnet152(pretrained=True)
modules=list(resnet152.children())[:-1]
resnet152=nn.Sequential(*modules)
for p in resnet152.parameters():
    p.requires_grad = False

img = torch.Tensor(3, 224, 224).normal_() # random image
img.unsqueeze_(0)
img_var = Variable(img) # assign it to a variable
features_var = resnet152(img_var) # get the output from the last hidden layer of the pretrained resnet
features = features_var.data # get the tensor out of the variable
print('features', features.size())

features torch.Size([1, 2048, 1, 1])


In [1]:
import os
import torch
import numpy as np
import nltk

glove_path = '../dataset_224/activitynet/glove.6B.300d.txt'
word2vec_path = '../dataset_224/activitynet/word2vec.pt'
vec2word_path = '../dataset_224/activitynet/vec2word.pt'
dataset_path = '../dataset_224/activitynet/m_data.pt'


In [114]:
# # First, you're going to need to import wordnet:
# from nltk.corpus import wordnet
 
# # Then, we're going to use the term "program" to find synsets like so:
# syns = wordnet.synsets("Javelin")
# print(syns)
# for item in syns:
# #     print(item.name().split('.')[0])
#     print(item.lemmas()[0].name())
#     print(item.definition())
#     print(item.examples())
#     break

[Synset('javelin.n.01'), Synset('javelin.n.02')]
javelin
an athletic competition in which a javelin is thrown as far as possible
[]


## Set of words in dataset

In [2]:
dataset = torch.load(dataset_path)
data_set = set()
for data_kind in dataset:
    for _, (_, value) in enumerate(dataset[data_kind].items()):
        captions = list(value.keys())
        for caption in captions:
            tokens = caption.split()
            for token in tokens:
                data_set.add(token)

In [9]:
print(len(data_set))

306


In [4]:
def word_in_dataset(word):
        return word in data_set

In [7]:
idx = 0
word2vec = {}
vec2word = {}

with open(glove_path, 'r') as file:
    for i, line in enumerate(file):
            line_list = line.split()
            word = line_list[0]
            if word_in_dataset(word):
                array_str = ' '.join(line_list[1:])
                word2vec.update({word:array_str})
                vec2word.update({array_str:word})
                       
    # Save both dictionary to .pt files.
    torch.save(word2vec, word2vec_path)
    torch.save(vec2word, vec2word_path)
    print('saved...')


saved...


In [6]:
dict1 = torch.load(word2vec_path)
dict2 = torch.load(vec2word_path)
print(len(dict1))
print(len(dict2))






160
160


### Truncate word2vec, vec2word to words present in dataset.